In [1]:
from RooPandasFunctions import PSequential,PColumn,PFilter,PRow,PProcessor,PProcRunner,PInitDir
import pandas as pd
from glob import glob
import ROOT
from ROOT import TH1F,TH2F,TLorentzVector,TFile,TCanvas,TLegend,gPad
from collections import OrderedDict
import numpy as np
import copy
import pyarrow as pa
import array
from optparse import OptionParser
import subprocess,os,sys
import time

Welcome to JupyROOT 6.22/00


In [2]:
#parser = OptionParser()
#
#parser.add_option('-p', '--nproc', metavar='F', type='string', action='store',
#                  default	=	'6',
#                  dest		=	'nproc',
#                  help		=	'nproc')
#
#parser.add_option('-n', '--njet', metavar='F', type='string', action='store',
#                  default	=	'3',
#                  dest		=	'njet',
#                  help		=	'njet')
#
#parser.add_option('-m', '--massrange', metavar='F', type='string', action='store',
#                  default	=	'0',
#                  dest		=	'massrange',
#                  help		=	'0,1,2,3,all')
#
#parser.add_option('-a', '--aeval', metavar='F', type='string', action='store',
#                  default	=	'95',
#                  dest		=	'aeval',
#                  help		=	'90,95,99')
#
#(options, args) = parser.parse_args()
#op_nproc=int(options.nproc)
#op_njet=int(options.njet)
#op_massrange=options.massrange
#op_aeval=options.aeval

In [3]:
op_nproc=int(6)
op_njet=int(3)
op_massrange="all"
op_aeval="99"
ntoys=0
quickrun=False
qcdonly=True
if quickrun:
    op_nproc=1

In [4]:
ROOT.gROOT.SetBatch(True)
ROOT.PyConfig.IgnoreCommandLineOptions = True

In [5]:
#this creates histos and  weights before any selection
class PreColumn():
    def __call__(self,df,EventInfo):
        EventInfo.eventcontainer["evweight"] = EventInfo.eventcontainer["lumi"]*EventInfo.eventcontainer["xsec"][EventInfo.dataset]/EventInfo.eventcontainer["nev"][EventInfo.dataset]
        df["Hists"]["logMSE_all"] = np.log(df["FatJet"]["iAEMSE"])
        df["Hists"]["weight"] *= EventInfo.eventcontainer["evweight"]
        #meh, should be added to  columnweights -- todo
        df["Hists"]["logMSE_all__weight"] = pd.Series(EventInfo.eventcontainer["evweight"], df["Hists"]["logMSE_all"].index, name="logMSE_all__weight")
        return df

In [6]:
#Select jetwise and eventwise. Exactly 4 jets with pt in region X, and all have sdmass in region Y
class KinematicSelection():
    def __init__(self,njet,ptcut,msdcut,htcut):
        self.ptcut=ptcut
        self.njet=njet
        self.msdcut=msdcut
        self.htcut=htcut
    def __call__(self,df,EventInfo):
        
        fjcutpt=(df["FatJet"]["pt"]>self.ptcut[0]) & (df["FatJet"]["pt"]<self.ptcut[1]) 
        df["FatJet"]=df["FatJet"][fjcutpt]
        C0=(df["FatJet"]["event"].count(level=0))==self.njet
        
        fjcuteta=(df["FatJet"]["eta"].abs()>0.0) &(df["FatJet"]["eta"].abs()<2.0) 
        df["FatJet"]=df["FatJet"][fjcuteta]
        C1=(df["FatJet"]["event"].count(level=0))==self.njet

        fjcutmass=(df["FatJet"]["msoftdrop"]>self.msdcut[0])&(df["FatJet"]["msoftdrop"]<self.msdcut[1])
        df["FatJet"]=df["FatJet"][fjcutmass]
        C2=(df["FatJet"]["event"].count(level=0))==self.njet

        allht = df["FatJet"]["pt"].sum(level=0)
        C3 = (allht > self.htcut[0]) & (allht < self.htcut[1])
        
        fjcut=fjcutpt&fjcutmass&fjcuteta
        C4=((fjcut).sum(level=0)>0)
   

        if (not(C0 & C1 & C2 & C3 & C4).any()):
            return None
        return (C0 & C1 & C2 & C3 & C4)

In [7]:
#Select DeltaR cut to make sure AK8 jets are separated
class KinematicSelectionDR():
    def __init__(self,njet,drcut):
        self.drcut=drcut
        self.njet=njet
    def __call__(self,df,EventInfo):    
        alldiscs=[]
        mindr = None
        for ijet in range(self.njet):
            #todo: find better way to check for nulls
            try:
                ijetphi=df["FatJet"]["phi"][:,ijet]
                ijeteta=df["FatJet"]["eta"][:,ijet]
            except:
                print ("ERR")
                print (df["FatJet"]["phi"])
                print (df["FatJet"]["eta"])
                return None
            drcutjet=None
            for jjet in range(self.njet):

                if ijet==jjet:
                    continue
            
                jjetphi=df["FatJet"]["phi"][:,jjet]
                jjeteta=df["FatJet"]["eta"][:,jjet]

                deta=(jjeteta-ijeteta).abs()
                dphi=(jjetphi-ijetphi).abs()
                dphi[dphi>3.1415]=2*3.1415-dphi
                dr=np.sqrt(dphi*dphi+deta*deta)
                curcond=dr>self.drcut

                if isinstance(drcutjet,type(None)):
                    drcutjet=curcond
                else:
                    drcutjet = drcutjet&(curcond)
                if isinstance(mindr,type(None)):
                    mindr = dr
                else:
                    mindr = pd.concat([mindr,dr]).min(level=0)
            alldiscs.append(drcutjet)
        
        
        df["Hists"]["mindr"]=mindr
        
        for iad,ad in enumerate(alldiscs):
            if iad==0:
                evdisc=ad
            else:
                evdisc=evdisc&ad
        #print("evd",evdisc)
        return ( evdisc )

In [8]:
#Create tight and loose jet tags
class MakeTags():
    def __init__(self,njet):
        self.njet=njet
    def __call__(self,df,EventInfo):

        cut90,cut95,cut99=-11.28,-10.74,-9.9
        logmse=np.log(df["FatJet"]["iAEMSE"])
        
        if op_aeval=="90":
            AEcut=cut90
        elif op_aeval=="95":
            AEcut=cut95
        elif op_aeval=="99":
            AEcut=cut99
        else:
            raise ValueError("Bad AE cut")

        njettight=((logmse>AEcut).sum(level=0))
        njetloose=((logmse<AEcut).sum(level=0))

        df["FatJet"]["tight"] = logmse>AEcut
        df["FatJet"]["loose"] = logmse<AEcut

        df["Hists"]["ht"]=df["FatJet"]["pt"].sum(level=0)
        df["Hists"]["njettight"] = njettight
        df["Hists"]["njetloose"] = njetloose

        return df

In [9]:
#project weights
class ColumnWeights():
    def __call__(self,df,EventInfo):
        keys=list(df["Hists"].keys())
        for hh in keys:
            if hh in ["njettight__njetloose","event","weight"]:
                continue
            if hh+"__weight" in df["Hists"]:
                continue
            df["Hists"][hh+"__weight"]=df["Hists"]["weight"]
            if (df["Hists"][hh].index.nlevels > df["Hists"]["weight"].index.nlevels )  :
                df["Hists"][hh]=df["Hists"][hh].droplevel(level=1)

            df["Hists"][hh+"__weight"] = df["Hists"][hh+"__weight"][df["Hists"][hh+"__weight"].index.isin(df["Hists"][hh].index)]
        
        df["Hists"]["njettight__njetloose__weight"]=df["Hists"]["njettight__weight"]
        return df

In [10]:
#make histograms to be used for creating the pass-to-fail ratio
class MakeHistsForBkg():
    def __init__(self,njet):
        self.njet=njet
    def __call__(self,df,EventInfo):
        bkgparam=EventInfo.eventcontainer["bkgparam"]
        for ijet in range(self.njet+1):
            for ptbin in bkgparam["pt"]:
                #tightreq=(df["Hists"]["njettight"]==1) | (df["Hists"]["njettight"]==0)
                tightreq=True
                abseta=df["FatJet"]["eta"].abs()
                #print("max eta: ",max(abseta))
                pt=df["FatJet"]["pt"]
                ptcut=(bkgparam["pt"][ptbin][0]<=pt)&(pt<bkgparam["pt"][ptbin][1])
                #print("ptcut: ",ptcut)
                regionstr="LT"+str(ijet)+str(njet-ijet)
                #print("regionstr: ",regionstr)
                try:
                    
                    df["Hists"]["etaT"+str(ijet)+"_"+ptbin]=abseta[df["FatJet"]["tight"]][ptcut][:,ijet]
                    df["Hists"]["etaT21"+str(ijet)+"_"+ptbin]=abseta[df["FatJet"]["tight"]][ptcut][:,ijet][df["Hists"]["njetloose"]==2][df["Hists"]["njettight"]==1]
                except Exception as e:
                    #print("Fail etaT")
                    #print(e)
                    pass
                try:
                    df["Hists"]["etaL"+str(ijet)+"_"+ptbin]=abseta[df["FatJet"]["loose"]][ptcut][:,ijet]
                    df["Hists"]["etaL21"+str(ijet)+"_"+ptbin]=abseta[df["FatJet"]["loose"]][ptcut][:,ijet][df["Hists"]["njetloose"]==2][df["Hists"]["njettight"]==1]
                    df["Hists"]["etaL30"+str(ijet)+"_"+ptbin]=abseta[df["FatJet"]["loose"]][ptcut][:,ijet][df["Hists"]["njetloose"]==3][df["Hists"]["njettight"]==0]
                except Exception as e:
                    #print("Fail etaL")
                    #print(e)
                    pass

            
            df["Hists"]["ht_"+regionstr]=df["Hists"]["ht"][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
            #df["Hists"]["mindr_"+regionstr]=df["Hists"]["mindr"][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
            
            for jjet in range(njet):
                df["Hists"]["pt"+str(jjet)+"_"+regionstr]=df["FatJet"]["pt"][:,jjet][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
                #df["Hists"]["ptTIGHT"+str(jjet)+"_"+regionstr]=df["FatJet"]["pt"][df["FatJet"]["tight"]][:,jjet][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
                #df["Hists"]["ptLOOSE"+str(jjet)+"_"+regionstr]=df["FatJet"]["pt"][df["FatJet"]["loose"]][:,jjet][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
                df["Hists"]["abseta"+str(jjet)+"_"+regionstr]=abseta[:,jjet][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
                #df["Hists"]["msd"+str(jjet)+"_"+regionstr]=df["FatJet"]["msoftdrop"][:,jjet][df["Hists"]["njettight"]==(njet-ijet)][df["Hists"]["njetloose"]==(ijet)]
        return df

In [11]:
#use pass-to-fail ratio created in step0 to predict background
#todo: Sometimes returns none -- look into
class BkgEst():
    
    def __init__(self,njet):
        self.njet=njet
    
    def prepdf(self,df):
        #args=[df["Hists"]["mindr"],df["Hists"]["ht"]]
        args=[df["Hists"]["ht"]]
        #print("len args: ",len(args))
        try:
            for ijet in range(self.njet):
                args.append(df["FatJet"]["pt"][:,ijet])
                args.append(df["FatJet"]["eta"][:,ijet].abs())
                args.append(df["FatJet"]["tight"][:,ijet])
                args.append(df["FatJet"]["loose"][:,ijet])
                #args.append(df["FatJet"]["msoftdrop"][:,ijet])
        except Exception as e:
            print (e)
            return None
        return args
    
    def __call__(self,args,EventInfo):
        
        bkgparam=EventInfo.eventcontainer["bkgparam"]
        RateHists=EventInfo.eventcontainer["RateHists"]
        RateHistsFULL=EventInfo.eventcontainer["RateHistsFULL"]
        #print(RateHistsFULL)
        #mindr=args[0]
        ht=args[0]
        pt=[]
        eta=[]
        tight=[]
        loose=[]
        ptTIGHT=[]
        ptLOOSE=[] 
        msd=[]
        
        for ijet in range(self.njet):
            pt.append(args[ijet*4+1])
            eta.append(args[ijet*4+2])
            tight.append(args[ijet*4+3])
            loose.append(args[ijet*4+4])
            #msd.append(args[ijet*5+5])
            #regionstr="LT"+str(ijet)+str(njet-ijet)
        
        nloosetrue=0
        for ll in loose:
            nloosetrue+=ll
        ntighttrue=0
        for tt in tight:
            ntighttrue+=tt
        
        #if ((nloose)==self.njet):
        
        maxbin=2**self.njet
        allregs=list(range(maxbin))
        allregs.reverse()
        Trate=[]
        Lrate=[]
        Trateup=[]
        Lrateup=[]   
        Tratedown=[]
        Lratedown=[] 
        jetebin=[] 
        jetptbin=[] 

        usefullrate=True

 
        Trateptjet=[]
        for ijet in range(self.njet):
            Trateptjet.append([])
            for iebin,ebin in enumerate(bkgparam["pt"]):
                Trateptjet[ijet].append(0.0)
                ptcut=(bkgparam["pt"][ebin][0]<=pt[ijet]<bkgparam["pt"][ebin][1])
                
                if ptcut:

                    if usefullrate:
                        etabin=RateHistsFULL["Rate"+ebin+"jet"+str(ijet)].FindBin(eta[ijet])
                        TRtemp=RateHistsFULL["Rate"+ebin+"jet"+str(ijet)].GetBinContent(etabin)
                        TRLtemp=RateHistsFULL["RateL"+ebin+"jet"+str(ijet)].GetBinContent(etabin)
                        TRtemperr=RateHistsFULL["Rate"+ebin+"jet"+str(ijet)].GetBinError(etabin)
                    else:
                        etabin=RateHists["Rate"+ebin].FindBin(eta[ijet])
                        TRtemp=RateHists["Rate"+ebin].GetBinContent(etabin)
                        TRLtemp=RateHists["RateL"+ebin+"jet"+str(ijet)].GetBinContent(etabin)
                        TRtemperr=RateHists["Rate"+ebin].GetBinError(etabin)

                    Trateptjet[ijet][iebin]=TRtemp
                    
                    if ntighttrue == 0:
                        Trate.append(TRtemp)
                        Lrate.append(TRLtemp)
                    else:
                        Trate.append(0.0)
                        Lrate.append(0.0)
                    #print("(pt) iebin",iebin)
                    #print("(eta) etabin",etabin)
                    jetptbin.append(iebin)                  
                    jetebin.append(etabin)
                    
                    
        weights=[0.0]*(self.njet+1)
        weightsT=[0.0]*(self.njet+1)
        weightsL=[0.0]*(self.njet+1)
        weights1=[0.0]*(self.njet+1)
        nweights=[0.0]*(self.njet+1)
        
        for ar in allregs:
            ntight=0
            nloose=0
            weight=1.0
            weightT=1.0
            weightL=1.0
            weight1=1.0
            for ibit,bit in enumerate(range(self.njet)):
                curbit=(ar>>bit)&1
                ntight+=curbit
                nloose+=(curbit==0)
            for ibit,bit in enumerate(range(self.njet)):
            
                curbit=(ar>>bit)&1

                #really not sure what the right one is.
                if curbit:
                    weight*=Trate[ibit] #is this nonsense?
                    #I use weight1 now.  Not sure though -- this is sum(Rloose)+sum(Rtight) but from LT21 so like the Rloose per jet is Rloose/2.  
                    #Based on the pt agreement, this is average of loose and tight for all jets.  I think there is a better way to do this  
                    weight1*=(float(ntight)*Trate[ibit]/1.0+float(nloose)*Lrate[ibit]/2.0)/float(nloose+ntight) #is this nonsense? 
                    weightT*=float(ntight)*Trate[ibit]/1.0 #is this nonsense?
                    weightL*= 1.0#is this nonsense?

                else: 
                    weight*=1.0-Trate[ibit] #is this nonsense?
                    weight1*=1.0 #is this nonsense?
                    weightT*=1.0 #is this nonsense?
                    weightL*=float(nloose)*Lrate[ibit]/2.0 #is this nonsense?
            #print
            weights1[self.njet-ntight]+=weight1
            weights[self.njet-ntight]+=weight
            weightsT[self.njet-ntight]+=weightT
            weightsL[self.njet-ntight]+=weightL
            nweights[self.njet-ntight]+=1.0
        weightLT=list((np.array(weightsL)+np.array(weightsT)))
        #print("----")
        #print(weights)
        #print(weights1)
        #print(weightsT)
        #print(weightsL)
        #print(weightLT)

        #print()
        allret=[]

        #This is the weighted average for ht.  Not sure if this makes sense, each jet has a probabilistic weight so maybe something like this
        #htw = 0.0
        #denom = 0.0
        #if ntighttrue==0:
        #    for ijet in range(njet):
        #        htw+=pt[ijet]*(Trate[ijet]+Lrate[ijet])
        #        denom+=(Trate[ijet]+Lrate[ijet])/3.0
        #    htw/=denom
        #print("ht, htw",ht,htw)

        for icweight,cweight in enumerate(weights1):
            
            #allret.append(mindr)
            #allret.append(cweight*EventInfo.eventcontainer["evweight"])
            
            allret.append(ht)
            allret.append(cweight*EventInfo.eventcontainer["evweight"])
              
            for jjet in range(njet):
                
                #pt of all events-- Events where jet==jjet is tight + Events where jet==jjet is loose
                #ht is something like sum(pt) with this weight averged over three jets?
                allret.append(pt[jjet])
                allret.append((Trate[jjet]+Lrate[jjet])*EventInfo.eventcontainer["evweight"])
                
                #pt of all tight events-- Events where jet==jjet is tight + 0
                #allret.append(pt[jjet])
                #allret.append(Trate[jjet]*EventInfo.eventcontainer["evweight"])

                #pt of all loose events--  0 + Events where jet==jjet is loose
                #allret.append(pt[jjet])
                #allret.append(Lrate[jjet]*EventInfo.eventcontainer["evweight"])
                
                allret.append(eta[jjet])
                allret.append((Trate[jjet]+Lrate[jjet])*EventInfo.eventcontainer["evweight"])
                
                #allret.append(msd[jjet])
                #allret.append(cweight*EventInfo.eventcontainer["evweight"])
                
                allret.append(jetptbin[jjet])
                allret.append(jetebin[jjet])
        
        #for jjet in range(njet):
        #    for iebin,ebin in enumerate(bkgparam["pt"]):
        #       
        #        allret.append(eta[jjet])
        #        allret.append(Trateptjet[jjet][iebin]*EventInfo.eventcontainer["evweight"])
        #        
        #        ptcut=(bkgparam["pt"][ebin][0]<=pt[jjet]<bkgparam["pt"][ebin][1])
        #        allret.append(eta[jjet])
        #        allret.append(ptcut*tight[jjet]*EventInfo.eventcontainer["evweight"])
        
        #print("len allret: ",len(allret))
        return (allret)

In [12]:
class MakeToys():
    def __init__(self,njet):
        self.njet=njet
    def __call__(self,df,EventInfo):
        bkgparam=EventInfo.eventcontainer["bkgparam"]
                
        maxbin=2**self.njet
        allregs=list(range(maxbin))
        allregs.reverse()

        for ijet in range(self.njet+1):

            regionstr="LT"+str(ijet)+str(njet-ijet) 
            
            allt=EventInfo.eventcontainer["toys"]

            allteval=[]
            
            for jjet in range(njet):
                allteval.append(allt[df["Hists"]["ptbin"+str(jjet)+"_"+regionstr],:,df["Hists"]["ebin"+str(jjet)+"_"+regionstr]])
                allteval[-1]=allteval[-1].squeeze()

            for tt in range(ntoys):

                sumem=np.zeros(df["Hists"]["ht"].shape)
                times=[]
                if True:

                    resampleT=[]
                    resampleL=[]

                    for jjet in range(njet):
               

                        resampleT.append(allteval[jjet][:,tt])              
                        resampleL.append(np.ones(resampleT[-1].shape))             
                        resampleL[-1]=(resampleL[-1]-resampleT[-1])
                                 
                
                    for ar in allregs:
                        ntight=0
                        for ibit,bit in enumerate(range(self.njet)):
                            curbit=(ar>>bit)&1
                            if curbit:
                                ntight+=1
                        if ntight!=(njet-ijet):
                            continue
                        weight=np.ones(resampleT[-1].shape)
                        for ibit,bit in enumerate(range(self.njet)):

                            curbit=(ar>>bit)&1
                            
                            if curbit:
                                weight*=resampleT[ibit]    
                            else:
                                weight*=resampleL[ibit]  
                        sumem+=weight*EventInfo.eventcontainer["evweight"]
                df["Hists"]["bkg_ht_toy"+str(tt)+"_"+regionstr+"__weight"]=pd.Series(sumem,index=df["Hists"]["ht"].index)
                df["Hists"]["bkg_ht_toy"+str(tt)+"_"+regionstr]=df["Hists"]["ht"]


        return df

In [13]:
chunklist=PInitDir("RooFlatFull")
for ds in chunklist:
        if qcdonly:
            if (ds.split("_")[0]!="QCD"):
                del chunklist[ds]
bkgparam={}

#three eta bins (probably overkill)
#bkgparam["eta"]={"E0":[0.0,0.4],"E1":[0.4,0.9],"E2":[0.9,1.3],"E3":[1.3,float("inf")]}

#swapping eta and pt
#bkgparam["pt"]={"pt0":[0,200],"pt1":[200,220],"pt2":[220,240],"pt3":[240,280],"pt4":[280,320],"pt5":[320,340],"pt6":[340,380],"pt7":[380,400],"pt8":[400,440],"pt9":[440,480],"pt10":[480,520],"pt11":[520,580],"pt12":[580,650],"pt13":[650,700],"pt14":[700,800],"pt15":[800,900],"pt16":[900,1000],"pt17":[1000,1100],"pt18":[1100,1200],"pt19":[1200,1300],"pt20":[1300,1500],"pt21":[1500,2000],"pt22":[2000,10000]}

bkgparam["pt"]={
"pt0":[0,200],
"pt1":[200,220],
"pt2":[220,240],
"pt3":[240,260],
"pt4":[260,280],
"pt5":[280,300],
"pt6":[300,320],
"pt7":[320,340],
"pt8":[340,360],
"pt9":[360,380],
"pt10":[380,400],
"pt11":[400,420],
"pt12":[420,440],
"pt13":[440,460],
"pt14":[460,480],
"pt15":[480,500],
"pt16":[500,520],
"pt17":[520,540],
"pt18":[540,560],
"pt19":[560,580],
"pt20":[580,600],
"pt21":[600,620],
"pt22":[620,650],
"pt23":[650,680],
"pt24":[680,710],
"pt25":[710,740],
"pt26":[740,800],
"pt27":[800,900],
"pt28":[900,1000],
"pt29":[1000,1100],
"pt30":[1100,1200],
"pt31":[1200,1300],
"pt32":[1300,1400],
"pt33":[1400,1500],
"pt34":[1500,1600],
"pt35":[1600,1700],
"pt36":[1700,1800],
"pt37":[1800,1900],
"pt38":[1900,2000],
"pt39":[2000,10000]
}



#fewer pt bins
#bkgparam["pt"]={"pt0":[0,200],"pt1":[200,240],"pt3":[240,320],"pt5":[320,380],"pt7":[380,440],"pt9":[440,520],"pt11":[520,650],"pt13":[650,800],"pt15":[800,1000],"pt17":[1000,1500],"pt19":[1500,2000],"pt20":[2000,10000]}


#todo: muon triggers a failure mode as sometimes events have no muons and no filter remo 
branchestoread={
                    #"Muon":["pt","eta","phi","mass"],
                    "FatJet":["pt","eta","phi","mass","msoftdrop","iAEMSE"],
                    "":["run","luminosityBlock","event"]
                    }

scalars=[""]
if op_massrange=="all":
    sdcut=[0.0,float("inf")]
else:
    #sdcuts=[[0.0,50.0],[50.0,100.0],[100.0,140.0],[140.0,200.0],[200.0,float("inf")]]
    sdcuts=[[0.0,50.0],[50.0,float("inf")]]
    sdcut=sdcuts[int(op_massrange)]

In [14]:
def MakeProc(njet,step,evcont):
    histostemp=OrderedDict  ([])
    if step==0:
        histostemp["logMSE_all"]=TH1F("logMSE_all","logMSE_all",100,-20.,0.)
        for ijet in range(njet+1):
            regionstr="LT"+str(ijet)+str(njet-ijet)
            #histostemp["mindr_"+regionstr]=TH1F("mindr_"+regionstr,"mindr_"+regionstr,100,0,4)
            histostemp["ht_"+regionstr]=TH1F("ht_"+regionstr,"ht_"+regionstr,700,0,7000)
            for jjet in range(njet):
                histostemp["pt"+str(jjet)+"_"+regionstr]=TH1F("pt"+str(jjet)+"_"+regionstr,"pt"+str(jjet)+"_"+regionstr,1000,0,10000)
                #histostemp["ptTIGHT"+str(jjet)+"_"+regionstr]=TH1F("ptTIGHT"+str(jjet)+"_"+regionstr,"ptTIGHT"+str(jjet)+"_"+regionstr,1000,0,10000)
                #histostemp["ptLOOSE"+str(jjet)+"_"+regionstr]=TH1F("ptLOOSE"+str(jjet)+"_"+regionstr,"ptLOOSE"+str(jjet)+"_"+regionstr,1000,0,10000)
                histostemp["abseta"+str(jjet)+"_"+regionstr]=TH1F("abseta"+str(jjet)+"_"+regionstr,"abseta"+str(jjet)+"_"+regionstr,50,0,2.5)
                #histostemp["msd"+str(jjet)+"_"+regionstr]=TH1F("msd"+str(jjet)+"_"+regionstr,"msd"+str(jjet)+"_"+regionstr,500,0,1000)

            for ebin in bkgparam["pt"]:
                    histostemp["etaL"+str(ijet)+"_"+ebin]=TH1F("etaL"+str(ijet)+"_"+ebin,"etaL"+str(ijet)+"_"+ebin,50,0,2.5)
                    histostemp["etaT"+str(ijet)+"_"+ebin]=TH1F("etaT"+str(ijet)+"_"+ebin,"etaT"+str(ijet)+"_"+ebin,50,0,2.5)
                    histostemp["etaL30"+str(ijet)+"_"+ebin]=TH1F("etaL30"+str(ijet)+"_"+ebin,"etaL30"+str(ijet)+"_"+ebin,50,0,2.5)
                    histostemp["etaT21"+str(ijet)+"_"+ebin]=TH1F("etaT21"+str(ijet)+"_"+ebin,"etaT21"+str(ijet)+"_"+ebin,50,0,2.5)
                    histostemp["etaL21"+str(ijet)+"_"+ebin]=TH1F("etaL21"+str(ijet)+"_"+ebin,"etaL21"+str(ijet)+"_"+ebin,50,0,2.5)
        myana=  [
                PColumn(PreColumn()),
                PFilter(KinematicSelection(njet,[200.0,float("inf")],sdcut,[1200.0,float("inf")])), 
                PFilter(KinematicSelectionDR(njet,1.1)),
                PColumn(MakeTags(njet)),
                PColumn(MakeHistsForBkg(njet)),
                PColumn(ColumnWeights()),
                ]

    if step==1:
        
        hpass=[]
        for ijet in range(njet+1):
            regionstr="LT"+str(ijet)+str(njet-ijet)
            
            #histostemp["bkg_mindr_"+regionstr]=TH1F("bkg_mindr_"+regionstr,"bkg_mindr_"+regionstr,100,0,4)
            #hpass.append(["Hists","bkg_mindr_"+regionstr])
            #hpass.append(["Hists","bkg_mindr_"+regionstr+"__weight"])
            
            histostemp["bkg_ht_"+regionstr]=TH1F("bkg_ht_"+regionstr,"bkg_ht_"+regionstr,700,0,7000)
            hpass.append(["Hists","bkg_ht_"+regionstr])
            hpass.append(["Hists","bkg_ht_"+regionstr+"__weight"])

            for jjet in range(njet):
                
                histostemp["bkg_pt"+str(jjet)+"_"+regionstr]=TH1F("bkg_pt"+str(jjet)+"_"+regionstr,"bkg_pt"+str(jjet)+"_"+regionstr,1000,0,10000)
                hpass.append(["Hists","bkg_pt"+str(jjet)+"_"+regionstr])
                hpass.append(["Hists","bkg_pt"+str(jjet)+"_"+regionstr+"__weight"])
                
                #histostemp["bkg_ptTIGHT"+str(jjet)+"_"+regionstr]=TH1F("bkg_ptTIGHT"+str(jjet)+"_"+regionstr,"bkg_ptTIGHT"+str(jjet)+"_"+regionstr,1000,0,10000)
                #hpass.append(["Hists","bkg_ptTIGHT"+str(jjet)+"_"+regionstr])
                #hpass.append(["Hists","bkg_ptTIGHT"+str(jjet)+"_"+regionstr+"__weight"])
                #
                #histostemp["bkg_ptLOOSE"+str(jjet)+"_"+regionstr]=TH1F("bkg_ptLOOSE"+str(jjet)+"_"+regionstr,"bkg_ptLOOSE"+str(jjet)+"_"+regionstr,1000,0,10000)
                #hpass.append(["Hists","bkg_ptLOOSE"+str(jjet)+"_"+regionstr])
                #hpass.append(["Hists","bkg_ptLOOSE"+str(jjet)+"_"+regionstr+"__weight"])
                
                histostemp["bkg_abseta"+str(jjet)+"_"+regionstr]=TH1F("bkg_abseta"+str(jjet)+"_"+regionstr,"bkg_abseta"+str(jjet)+"_"+regionstr,50,0,2.5)
                hpass.append(["Hists","bkg_abseta"+str(jjet)+"_"+regionstr])
                hpass.append(["Hists","bkg_abseta"+str(jjet)+"_"+regionstr+"__weight"])
                
                #histostemp["bkg_msd"+str(jjet)+"_"+regionstr]=TH1F("bkg_msd"+str(jjet)+"_"+regionstr,"bkg_msd"+str(jjet)+"_"+regionstr,500,0,1000)
                #hpass.append(["Hists","bkg_msd"+str(jjet)+"_"+regionstr])
                #hpass.append(["Hists","bkg_msd"+str(jjet)+"_"+regionstr+"__weight"])            
                
                hpass.append(["Hists","ptbin"+str(jjet)+"_"+regionstr])
                hpass.append(["Hists","ebin"+str(jjet)+"_"+regionstr])
         
            for itoy in range(ntoys):
                histostemp["bkg_ht_toy"+str(itoy)+"_"+regionstr]=TH1F("bkg_ht_toy"+str(itoy)+"_"+regionstr,"bkg_ht_toy"+str(itoy)+"_"+regionstr,700,0,7000)         
        
        #for ijet in range(njet):
        #    for ebin in bkgparam["pt"]:
        #        histostemp["bkg_etaT"+str(ijet)+"_"+ebin]=TH1F("bkg_etaT"+str(ijet)+"_"+ebin,"bkg_etaT"+str(ijet)+"_"+ebin,50,0,2.5)
        #        hpass.append(["Hists","bkg_etaT"+str(ijet)+"_"+ebin])   
        #        hpass.append(["Hists","bkg_etaT"+str(ijet)+"_"+ebin+"__weight"])  
        #
        #        histostemp["CCptT"+str(ijet)+"_"+ebin]=TH1F("CCptT"+str(ijet)+"_"+ebin,"CCptT"+str(ijet)+"_"+ebin,1000,0,10000)
        #        hpass.append(["Hists","CCptT"+str(ijet)+"_"+ebin])   
        #        hpass.append(["Hists","CCptT"+str(ijet)+"_"+ebin+"__weight"])  
            
        print("len(hpass)",len(hpass))        
                    
        myana=  [
                PColumn(PreColumn()),
                PFilter(KinematicSelection(njet,[200.0,float("inf")],sdcut,[1200.0,float("inf")])),     
                PFilter(KinematicSelectionDR(njet,1.1)),
                PColumn(MakeTags(njet)),
                PRow(hpass,BkgEst(njet)),
                #PColumn(MakeToys(njet)),
                PColumn(ColumnWeights()),
                ]
    for hist in histostemp:
        histostemp[hist].Sumw2() 
        
    histos= {}
    for ds in chunklist:
        if quickrun:
            chunklist[ds]=chunklist[ds][:1]
        #chunklist[ds]=chunklist[ds][:12]
        histos[ds]=copy.deepcopy(histostemp)

    return PProcessor(chunklist,histos,branchestoread,myana,eventcontainer=evcont,atype="flat",scalars=scalars)

In [15]:
njet=op_njet
evcont={"lumi":(1000.0*137.65),"xsec":{"WgWg":1.0,"TT":1.0,"QCD_HT1500to2000":101.8,"QCD_HT1000to1500":1005.0,"QCD_HT2000toInf":20.54},"nev":{"WgWg":18000.0,"TT":305963.0,"QCD_HT1500to2000":10655313.0,"QCD_HT1000to1500":12660521.0,"QCD_HT2000toInf":4980828.0}}
evcont["bkgparam"]=bkgparam

In [16]:
#Step0:make hists for pass-to-fail ratio
proc = MakeProc(njet,0,evcont)
nproc=op_nproc
Mproc=PProcRunner(proc,nproc)
returndf=Mproc.Run()

Dataset:QCD_HT1000to1500 Process:3:   0%|          | 0/6 [00:00<?, ?it/s]

Running process over 6 processors


Dataset:QCD_HT2000toInf Process:2: 100%|██████████| 4/4 [12:30<00:00, 187.50s/it]


Done
Done
Total time 1883.8212909698486


In [17]:
#Print MSE quantilles
#for rr in returndf:
#    if  "logMSE_all" in returndf[rr]:
#        print  (rr ,"cut90",returndf[rr]["logMSE_all"].quantile(0.90))
#        print  (rr ,"cut95",returndf[rr]["logMSE_all"].quantile(0.95))
#        print  (rr ,"cut99",returndf[rr]["logMSE_all"].quantile(0.99))

In [18]:
ratehistos=copy.deepcopy(proc.hists)
qcdnames = ["QCD_HT1000to1500","QCD_HT1500to2000","QCD_HT2000toInf"]

In [19]:
#Make pass-to-fail ratio TR(pt,eta)
THists={}
LTHists={}
LHists={}
ALLHists={}

THistsFULL={}
LTHistsFULL={}
LHistsFULL={}

print("START")

bins=array.array('d',[0,0.4,0.9,1.3,1.5,1.7,1.9,2.1,2.3,2.5])

for ijet in range(njet):
    for qcdname in qcdnames:
        QCDhists=ratehistos[qcdname]
        for curhist in QCDhists:
            #print(curhist)
            if curhist[:7] == "etaL30"+str(ijet):
                print("curhist: ",curhist)
                Lstring=curhist
                Tstring=curhist.replace("etaL30"+str(ijet),"etaT21"+str(ijet))
                LTstring=curhist.replace("etaL30"+str(ijet),"etaL21"+str(ijet))
                
                paramstr=Lstring.split("_")[-2]
                paramstrwjet=Lstring.split("_")[-1]+"jet"+str(ijet)

                curhistL=QCDhists[Lstring]
                curhistT=QCDhists[Tstring]
                curhistLT=QCDhists[LTstring]
                print("Lstring",Lstring,curhistL.Integral())
                print("Tstring",Tstring,curhistT.Integral())

                if not(paramstr in THists):
                    THists[paramstr]=copy.deepcopy(curhistT)
                    LTHists[paramstr]=copy.deepcopy(curhistLT)
                    LHists[paramstr]=copy.deepcopy(curhistL)
                else:
                    THists[paramstr].Add(curhistT)
                    LTHists[paramstr].Add(curhistLT)
                    LHists[paramstr].Add(curhistL)

                if not(paramstrwjet in THistsFULL):
                    THistsFULL[paramstrwjet]=copy.deepcopy(curhistT)
                    LTHistsFULL[paramstrwjet]=copy.deepcopy(curhistLT)
                    LHistsFULL[paramstrwjet]=copy.deepcopy(curhistL)
                else:
                    THistsFULL[paramstrwjet].Add(curhistT)
                    LTHistsFULL[paramstrwjet].Add(curhistLT)
                    LHistsFULL[paramstrwjet].Add(curhistL)
     
                            
for tth in THists:
    THists[tth]=THists[tth].Rebin(len(bins)-1,THists[tth].GetName()+"TEMP",bins)    
for tth in LTHists:
    LTHists[tth]=LTHists[tth].Rebin(len(bins)-1,LTHists[tth].GetName()+"TEMP",bins)                    
for tth in LHists:
    LHists[tth]=LHists[tth].Rebin(len(bins)-1,LHists[tth].GetName()+"TEMP",bins)                    

for tth in THistsFULL:
    THistsFULL[tth]=THistsFULL[tth].Rebin(len(bins)-1,THistsFULL[tth].GetName()+"TEMP",bins)   
for tth in LTHistsFULL:
    LTHistsFULL[tth]=LTHistsFULL[tth].Rebin(len(bins)-1,LTHistsFULL[tth].GetName()+"TEMP",bins)                   
for tth in LHistsFULL:
    LHistsFULL[tth]=LHistsFULL[tth].Rebin(len(bins)-1,LHistsFULL[tth].GetName()+"TEMP",bins)     

START
curhist:  etaL300_pt0
Lstring etaL300_pt0 0.0
Tstring etaT210_pt0 0.0
curhist:  etaL300_pt1
Lstring etaL300_pt1 0.0
Tstring etaT210_pt1 0.0
curhist:  etaL300_pt2
Lstring etaL300_pt2 0.0
Tstring etaT210_pt2 0.0
curhist:  etaL300_pt3
Lstring etaL300_pt3 0.0
Tstring etaT210_pt3 0.0
curhist:  etaL300_pt4
Lstring etaL300_pt4 0.0
Tstring etaT210_pt4 0.0
curhist:  etaL300_pt5
Lstring etaL300_pt5 0.0
Tstring etaT210_pt5 0.0
curhist:  etaL300_pt6
Lstring etaL300_pt6 0.0
Tstring etaT210_pt6 0.0
curhist:  etaL300_pt7
Lstring etaL300_pt7 0.0
Tstring etaT210_pt7 0.0
curhist:  etaL300_pt8
Lstring etaL300_pt8 0.0
Tstring etaT210_pt8 0.0
curhist:  etaL300_pt9
Lstring etaL300_pt9 0.0
Tstring etaT210_pt9 0.0
curhist:  etaL300_pt10
Lstring etaL300_pt10 0.0
Tstring etaT210_pt10 0.0
curhist:  etaL300_pt11
Lstring etaL300_pt11 4851.473693847656
Tstring etaT210_pt11 10.926742553710938
curhist:  etaL300_pt12
Lstring etaL300_pt12 28092.654846191406
Tstring etaT210_pt12 196.68136596679688
curhist:  etaL30

curhist:  etaL302_pt0
Lstring etaL302_pt0 0.0
Tstring etaT212_pt0 0.0
curhist:  etaL302_pt1
Lstring etaL302_pt1 601375.8793945312
Tstring etaT212_pt1 2786.319351196289
curhist:  etaL302_pt2
Lstring etaL302_pt2 587706.498046875
Tstring etaT212_pt2 2644.271697998047
curhist:  etaL302_pt3
Lstring etaL302_pt3 559154.8818359375
Tstring etaT212_pt3 2808.172836303711
curhist:  etaL302_pt4
Lstring etaL302_pt4 547605.2954101562
Tstring etaT212_pt4 3026.7076873779297
curhist:  etaL302_pt5
Lstring etaL302_pt5 533837.583984375
Tstring etaT212_pt5 3201.5355682373047
curhist:  etaL302_pt6
Lstring etaL302_pt6 493648.97216796875
Tstring etaT212_pt6 2961.147232055664
curhist:  etaL302_pt7
Lstring etaL302_pt7 415118.37109375
Tstring etaT212_pt7 2152.5682830810547
curhist:  etaL302_pt8
Lstring etaL302_pt8 317662.6162109375
Tstring etaT212_pt8 1901.2532043457031
curhist:  etaL302_pt9
Lstring etaL302_pt9 226041.74243164062
Tstring etaT212_pt9 1376.7695617675781
curhist:  etaL302_pt10
Lstring etaL302_pt10 1

In [20]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

RateHists=OrderedDict([])
canvrate=TCanvas("canvrate","canvrate",700,500)
color=1

alltoys=[]
print("LHists",LHists)
for RH in LHists:
    #print(RH)
    #print(THists[RH].Integral())
    #print(LHists[RH].Integral())
    
    RateHists["Rate"+RH]=copy.deepcopy(THists[RH])
    RateHists["Rate"+RH].Divide(RateHists["Rate"+RH],LHists[RH])#,1.0,1.0,"B")

    RateHists["RateL"+RH]=copy.deepcopy(LTHists[RH])
    RateHists["RateL"+RH].Divide(RateHists["RateL"+RH],LHists[RH])#,1.0,1.0,"B")

    means = []
    errs = []
    toys = []
    for xbin in range(RateHists["Rate"+RH].GetXaxis().GetNbins()+1):
        means.append(RateHists["Rate"+RH].GetBinContent(xbin))
        errs.append(RateHists["Rate"+RH].GetBinError(xbin))

    curtoys=[]
    for tt in range(ntoys):
        curtoys.append(np.random.normal(means,errs))
    #print(curtoys[0].shape)
    alltoys.append(curtoys)
    #print (curtoys)
    #print("Rate"+RH)
    RateHists["Rate"+RH].SetLineColor(color)
    RateHists["Rate"+RH].SetMarkerColor(color)
    RateHists["Rate"+RH].Draw("same")
    color+=1
    
RateHistsFULL=OrderedDict([])
    
for RH in LHistsFULL:
 
    RateHistsFULL["Rate"+RH]=copy.deepcopy(THistsFULL[RH])
    RateHistsFULL["Rate"+RH].Divide(RateHistsFULL["Rate"+RH],LHistsFULL[RH])#,1.0,1.0,"B")

    RateHistsFULL["RateL"+RH]=copy.deepcopy(LTHistsFULL[RH])
    RateHistsFULL["RateL"+RH].Divide(RateHistsFULL["RateL"+RH],LHistsFULL[RH])#,1.0,1.0,"B")

    

canvrate.Print('plots/Trate.png', 'png')

evcont["RateHists"]=RateHists
evcont["RateHistsFULL"]=RateHistsFULL

evcont["toys"]=np.array(alltoys)
#print(evcont["toys"].shape)

LHists {'etaL300': <cppyy.gbl.TH1F object at 0x5576c4c4da60>, 'etaL301': <cppyy.gbl.TH1F object at 0x5576c4b98320>, 'etaL302': <cppyy.gbl.TH1F object at 0x5576c4c4ee80>}


Info in <TCanvas::Print>: png file plots/Trate.png has been created


In [21]:
#Step1:use pass-to-fail ratio to predict background
proc = MakeProc(njet,1,evcont)
Mproc=PProcRunner(proc,nproc)
returndf=Mproc.Run()

len(hpass) 80
Running process over 6 processors


Dataset:QCD_HT2000toInf Process:2: 100%|██████████| 4/4 [01:44<00:00, 26.19s/it]


Done
Done
Total time 311.7850306034088


In [32]:
histos=copy.deepcopy(proc.hists)


htosum={}
htosum["QCD"]=["QCD_HT1500to2000","QCD_HT1000to1500","QCD_HT2000toInf"]

histdicts=[histos,ratehistos]
for hdict in histdicts:
        for curh in htosum:
            hdict[curh]={}
            for var in hdict[htosum[curh][0]]:
                for curhsum in htosum[curh]:
                        if  var in hdict[curh]:
                                hdict[curh][var].Add(hdict[curhsum][var])
                        else:
                                hdict[curh][var] = copy.deepcopy(hdict[curhsum][var])
                                hdict[curh][var].SetName(hdict[curhsum][var].GetName().replace(curhsum,curh))
                                hdict[curh][var].SetTitle(hdict[curhsum][var].GetName().replace(curhsum,curh))

In [33]:
#Plot ht
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
output = TFile("FromFlatPandas_AE"+op_aeval+"_M"+op_massrange+"_Njet"+str(op_njet)+"_ntoys"+str(ntoys)+".root","recreate")
output.cd()

for RHtext in RateHists:
    RateHists[RHtext].Write("TRate"+RHtext)
for RHtext in RateHistsFULL:
    RateHistsFULL[RHtext].Write("TRate"+RHtext)
for ds in ratehistos:
    for var in ratehistos[ds]:
            ratehistos[ds][var].Write(ds+"__"+var)

#if ntoys>0:
#    for ds in histos:
#
#        canvtoys=TCanvas("httoys"+ds,"httoys"+ds,700,500)
#        canvtoyspread=TCanvas("httoyspread"+ds,"httoyspread"+ds,700,500)
#
#        histoiter=list(range(njet+1))
#        histoiter.reverse()
#        histoiter.pop(0)    
#        for ijet in histoiter:
#                
#                regionstr="LT"+str(ijet)+str(njet-ijet)
#
#                bkgname="bkg_ht_"+regionstr
#
#                dataname="ht_"+regionstr
#                canvtoys.cd()
#                bindists=[]
#
#                for itoy in  range(ntoys):
#                    bindists.append([])
#                    for xbin in range(histos[ds]["bkg_ht_toy"+str(itoy)+"_"+regionstr].GetXaxis().GetNbins()):
#                        bindists[-1].append(histos[ds]["bkg_ht_toy"+str(itoy)+"_"+regionstr].GetBinContent(xbin))
#                    if itoy==0:
#                        histos[ds]["bkg_ht_toy"+str(itoy)+"_"+regionstr].Draw("hist")  
#                    else:
#                        histos[ds]["bkg_ht_toy"+str(itoy)+"_"+regionstr].Draw("samehist") 
#                histos[ds]["bkg_ht_"+regionstr].Draw("same") 
#                binarr=np.array(bindists)
#                totoy=binarr.shape[0]
#                totbins=binarr.shape[1]
#                histos[ds]["bkg_ht_toyspread_"+regionstr]=copy.deepcopy(histos[ds][bkgname])
#                histos[ds]["bkg_ht_toyspread_"+regionstr].SetName("bkg_ht_toyspread_"+regionstr)
#                histos[ds]["bkg_ht_toyspread_"+regionstr].SetTitle("bkg_ht_toyspread_"+regionstr)
#                for ibin in range(totbins):
#                    #print(binarr[:,ibin].std())
#                    histos[ds]["bkg_ht_toyspread_"+regionstr].SetBinContent(ibin,binarr[:,ibin].mean())
#                    histos[ds]["bkg_ht_toyspread_"+regionstr].SetBinError(ibin,binarr[:,ibin].std())
#                canvtoyspread.cd()
#                histos[ds]["bkg_ht_toyspread_"+regionstr].Draw("same")
#    canvtoys.Write()    
#    canvtoys.Print('plots/httoys'+ds+'.png', 'png')  
#    canvtoys.Print('plots/httoys'+ds+'.root', 'root')  
#    image = mpimg.imread('plots/httoys'+ds+'.png')
#
#    #print("toys")

In [2]:
tocanv={"ht":[2,[0,5000]],"pt0":[2,[0,3000]],"pt1":[2,[0,3000]],"pt2":[2,[0,3000]],"ptT0":[2,[0,3000]],"ptT1":[2,[0,3000]],"ptT2":[2,[0,3000]],"ptL0":[2,[0,3000]],"ptL1":[2,[0,3000]],"ptL2":[2,[0,3000]]}

for tc in tocanv:
        print("tc: ",tc)
        for ds in histos:
            for var in histos[ds]:
                    histos[ds][var].Write(ds+"__"+var)
                    print(ds,var,histos[ds][var].Integral())
            rebinval=tocanv[tc][0]
            xrangeval=tocanv[tc][1]
            #print(histos[ds])
            canv=TCanvas(tc+ds,tc+ds,700,500)
            main = ROOT.TPad("main", "main", 0, 0.3, 1, 1)
            sub = ROOT.TPad("sub", "sub", 0, 0, 1, 0.3)

            main.SetLeftMargin(0.16)
            main.SetRightMargin(0.05)
            main.SetTopMargin(0.11)
            main.SetBottomMargin(0.0)

            sub.SetLeftMargin(0.16)
            sub.SetRightMargin(0.05)
            sub.SetTopMargin(0)
            sub.SetBottomMargin(0.3)

            main.Draw()
            sub.Draw()
            #canvrat=TCanvas("htrat"+ds,"htrat"+ds,700,500)
            gPad.SetLeftMargin(0.12)
            leg = TLegend(0.65, 0.55, 0.84, 0.84)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            histoiter=list(range(njet+1))
            histoiter.reverse()
            histoiter.pop(0)
            allrat=[]

            for ijet in histoiter:
                    regionstr="LT"+str(ijet)+str(njet-ijet)
                    if tc[0:2]=="pt":
  
                        if regionstr!="LT21":
                                continue

                    bkgname="bkg_"+tc+"_"+regionstr
                    dataname=tc+"_"+regionstr
                    color=ijet+1



                    ratehistos[ds][dataname].SetLineColor(color)
                    ratehistos[ds][dataname].SetTitle(";"+tc+"(GeV);events")
                    ratehistos[ds][dataname].SetStats(0) 
                    ratehistos[ds][dataname].Rebin(rebinval) 

                    ratehistos[ds][dataname].GetXaxis().SetRangeUser(xrangeval[0],xrangeval[1])

                    histos[ds][bkgname].SetLineColor(color)
                    histos[ds][bkgname].Rebin(rebinval) 

                    histos[ds][bkgname].GetXaxis().SetRangeUser(xrangeval[0],xrangeval[1])

                    main.cd()

                    ratehistos[ds][dataname].GetXaxis().SetTitleSize (0.06)
                    ratehistos[ds][dataname].GetXaxis().SetLabelSize (0.05)
                    ratehistos[ds][dataname].GetYaxis().SetTitleSize (0.06)
                    ratehistos[ds][dataname].GetYaxis().SetLabelSize (0.05)
                    ratehistos[ds][dataname].Draw("same")   
                    histos[ds][bkgname].Draw("histsame") 
                    
                    
                    leg.AddEntry(histos[ds][bkgname],ds+regionstr+"bkg","L")
                    leg.AddEntry(ratehistos[ds][dataname],ds+regionstr,"LE")

                    sub.cd()
                    allrat.append(copy.deepcopy(ratehistos[ds][dataname]) )
                    allrat[-1].Divide(histos[ds][bkgname])


                    allrat[-1].GetYaxis().SetRangeUser(0.5,1.5)
                    allrat[-1].SetTitle(";"+tc+"(GeV);")
                    allrat[-1].GetXaxis().SetTitleSize (0.12)
                    allrat[-1].GetXaxis().SetLabelSize (0.09)

                    allrat[-1].GetYaxis().SetTitleSize (0.12)
                    allrat[-1].GetYaxis().SetLabelSize (0.09)
                    allrat[-1].GetXaxis().SetRangeUser(xrangeval[0],xrangeval[1])


                    print("--Fit--",tc,regionstr)
                    if regionstr=="LT21":
                        allrat[-1].Fit("pol1")

                    line2=ROOT.TLine(xrangeval[0],1.0,xrangeval[1],1.0)
                    line2.SetLineColor(0)
                    line1=ROOT.TLine(xrangeval[0],1.0,xrangeval[1],1.0)
                    line1.SetLineStyle(2)

                    allrat[-1].Draw("histesame") 
                    line2.Draw()
                    line1.Draw()
            main.cd()
            leg.Draw()
            main.SetLogy()
            canv.Write()
            canv.Print('plots/'+tc+ds+'.png', 'png')
            canv.Print('plots/'+tc+ds+'.root', 'root')
            print(ds)

tc:  ht


NameError: name 'histos' is not defined

In [36]:
output.Close()